In [1]:
# import sys
# STATA_SYSDIR =  "/Applications/StataNow"
# sys.path.append(f'{STATA_SYSDIR}/utilities')

from pyutils.econometrics import stata
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'y': [1,2,3,4,5,6,7,8,9,10],
    'x1': [2,3,3,5,6,7,8,9,10,11],
    'x2': [5,3,6,3,7,4,8,1,9,0],
    'z': [10,9,8,7,6,5,4,3,2,1],
    'fe': [1,1,1,1,1,2,2,2,2,2],
    "dummy": ["a","a","a","a","a","b","b","b","b","b"]
})  

In [3]:
from pystata import stata # type: ignore

stata.pdataframe_to_data(data, force=True)

stata.run(f"encode dummy, generate(dummy_encoded)")

stata.run(f'reghdfe y x1 x2 dummy_encoded, absorb(fe) vce(robust)')

(MWFE estimator converged in 1 iterations)
note: dummy_encoded is probably collinear with the fixed effects (all partialle
> d-out values are close to zero; tol = 1.0e-09)

HDFE Linear regression                            Number of obs   =         10
Absorbing 1 HDFE group                            F(   2,      6) =     174.18
                                                  Prob > F        =     0.0000
                                                  R-squared       =     0.9908
                                                  Adj R-squared   =     0.9863
                                                  Within R-sq.    =     0.9622
                                                  Root MSE        =     0.3548

------------------------------------------------------------------------------
             |               Robust
           y | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------


In [5]:
from pyutils.econometrics.stata import reghdfe

result = reghdfe('y', ['x1', 'x2',"dummy"], 'fe', 'robust', data)


(MWFE estimator converged in 1 iterations)
note: dummy_encoded is probably collinear with the fixed effects (all partialle
> d-out values are close to zero; tol = 1.0e-09)

HDFE Linear regression                            Number of obs   =         10
Absorbing 1 HDFE group                            F(   2,      6) =     174.18
                                                  Prob > F        =     0.0000
                                                  R-squared       =     0.9908
                                                  Adj R-squared   =     0.9863
                                                  Within R-sq.    =     0.9622
                                                  Root MSE        =     0.3548

------------------------------------------------------------------------------
             |               Robust
           y | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------


/Users/marcdelabarrera/IESE Dropbox/Marc de la Barrera/repos/pyutils/src/pyutils/econometrics/stata/main.py:34: RuntimeWarning: invalid value encountered in sqrt
  "se": np.diag(np.sqrt(ereturn["e(V)"])),


In [5]:
from pyutils.econometrics.stata import reghdfe

result = reghdfe('y', ['x1', 'x2',"dummy"], 'fe', 'robust', data)
print(result)

SystemError: string variables not allowed in varlist;
dummy is a string variable
r(109);


In [5]:
print(result.coeftable)

           coef        se     t-stat   p-value
x1     0.964256  0.055810  17.277418  0.000000
x2     0.013460  0.019639   0.685367  0.493112
_cons -0.733156  0.409917  -1.788549  0.073688


In [39]:
print(result)

{'e(N)': 10.0, 'e(rank)': 2.0, 'e(df_r)': 6.0, 'e(df_m)': 2.0, 'e(tss)': 82.5, 'e(tss_within)': 20.000000000000004, 'e(rss)': 0.7552531219621622, 'e(mss)': 81.74474687803784, 'e(rmse)': 0.3547894027828721, 'e(F)': 174.18309472595158, 'e(ll)': -1.2729482418592664, 'e(ll_0)': -17.655121234846455, 'e(r2)': 0.9908454167034889, 'e(r2_within)': 0.9622373439018919, 'e(r2_a)': 0.9862681250552334, 'e(r2_a_within)': 0.9496497918691892, 'e(sumweights)': 10.0, 'e(ic)': 1.0, 'e(converged)': 1.0, 'e(report_constant)': 1.0, 'e(N_full)': 10.0, 'e(num_singletons)': 0.0, 'e(drop_singletons)': 1.0, 'e(df_a_nested)': 0.0, 'e(df_a_redundant)': 0.0, 'e(df_a_initial)': 2.0, 'e(df_a)': 2.0, 'e(N_hdfe_extended)': 1.0, 'e(N_hdfe)': 1.0, 'e(_r_z_abs__CL)': '|t|', 'e(_r_z__CL)': 't', 'e(extended_absvars)': 'fe', 'e(absvars)': 'fe', 'e(title2)': 'Absorbing 1 HDFE group', 'e(dofmethod)': 'pairwise clusters continuous', 'e(vce)': 'robust', 'e(vcetype)': 'Robust', 'e(title)': 'HDFE Linear regression', 'e(indepvars)':

In [44]:
result['e(cmdline)']

'reghdfe y x1 x2, absorb(fe) vce(robust)'

In [43]:
from scipy.stats import norm
coeftable = pd.DataFrame({"coef": result["e(b)"].flatten(),
              "se": np.diag(np.sqrt(result["e(V)"])),
              }, index = result["e(indepvars)"].split())
coeftable["t-stat"] = coeftable["coef"] / coeftable["se"]

coeftable["p-value"] = 2 * (1 - norm.cdf(np.abs(coeftable["t-stat"])))

coeftable

/var/folders/y1/yl45ltr524x8qrq55m3qvcww0000gn/T/ipykernel_49549/2555000240.py:3: RuntimeWarning: invalid value encountered in sqrt
  "se": np.diag(np.sqrt(result["e(V)"])),


,coef,se,t-stat,p-value
x1,0.964256,0.055810,17.277418,0.000000
x2,0.013460,0.019639,0.685367,0.493112
_cons,-0.733156,0.409917,-1.788549,0.073688


In [5]:
result["e(b)"]

array([[ 1.00000000e+00, -3.70298833e-17, -1.00000000e+00]])

In [13]:
from pystata import stata
import pandas as pd
def ivreg(estimator:str, depvar:str, exog:list[str], endog:str, instruments:list[str], data:pd.DataFrame):
    stata.pdataframe_to_data(data, force=True)
    stata.run(f'ivregress {estimator} {depvar} {" ".join(exog)} ({endog} = {" ".join(instruments)})')
    return  stata.get_ereturn()

In [14]:
data = pd.DataFrame({
    'y': [1,2,3,4,5,6,7,8,9,10],
    'x1': [2,3,4,5,6,7,8,9,10,11],
    'x2': [5,3,6,2,7,4,8,1,9,0],
    'z': [10,9,8,7,6,5,4,3,2,1],
    'fe': [1,1,1,1,1,2,2,2,2,2]
})  

# result = ivreg('2sls', 'y', ['x1', 'x2'], 'x1', ['z'], data)
# print(result)

In [15]:
#stata.run('ssc install ftools, replace')

checking ftools consistency and verifying not already installed...
all files already exist and are up to date.


In [16]:
#stata.run('ssc install reghdfe, replace')

In [18]:
stata.run('ssc install require, replace')


checking require consistency and verifying not already installed...
installing into /Users/marcdelabarrera/Library/Application Support/Stata/ado/pl
> us/...
installation complete.


In [4]:
stata.pdataframe_to_data(data, force=True)
stata.run('xtset fe')
stata.run(f'xtreg y x1 x2, fe')
stata.get_ereturn()


Panel variable: fe (balanced)

Fixed-effects (within) regression               Number of obs     =         10
Group variable: fe                              Number of groups  =          2

R-squared:                                      Obs per group:
     Within  = 1.0000                                         min =          5
     Between = 1.0000                                         avg =        5.0
     Overall = 1.0000                                         max =          5

                                                F(2, 6)           =          .
corr(u_i, Xb) = -0.8704                         Prob > F          =          .

------------------------------------------------------------------------------
           y | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          x1 |          1          .        .       .            .           .
          x2 |   1.24e-17        

{'e(r2_c)': 1.0,
 'e(F_f)': 8.98846567431158e+307,
 'e(p_f)': 8.98846567431158e+307,
 'e(Tbar)': 5.0,
 'e(Tcon)': 1.0,
 'e(g_min)': 5.0,
 'e(g_avg)': 5.0,
 'e(g_max)': 5.0,
 'e(N_g)': 2.0,
 'e(k_absorb)': 2.0,
 'e(rc)': 0.0,
 'e(ui)': 1.444485061989073e-15,
 'e(sigma_u)': 1.444485061989073e-15,
 'e(corr)': -0.8703882797784892,
 'e(r2_o)': 1.0,
 'e(r2_b)': 1.0,
 'e(sigma_e)': 0.0,
 'e(sigma)': 1.444485061989073e-15,
 'e(rho)': 1.0,
 'e(rank)': 0.0,
 'e(p)': 8.98846567431158e+307,
 'e(r2_w)': 1.0,
 'e(df_m)': 3.0,
 'e(df_b)': 2.0,
 'e(ll_0)': -17.655121234846455,
 'e(tss)': 82.5,
 'e(df_a)': 1.0,
 'e(r2_a)': 1.0,
 'e(rss)': 0.0,
 'e(mss)': 20.0,
 'e(rmse)': 0.0,
 'e(r2)': 1.0,
 'e(F)': 8.98846567431158e+307,
 'e(df_r)': 6.0,
 'e(sum_w)': 10.0,
 'e(N)': 10.0,
 'e(cmdline)': 'xtreg y x1 x2, fe',
 'e(estat_cmd)': 'xtreg_estat',
 'e(_r_z_abs__CL)': '|t|',
 'e(_r_z__CL)': 't',
 'e(cmd)': 'xtreg',
 'e(marginsprop)': 'nolinearize',
 'e(marginsnotok)': 'E U UE SCore STDP XBU XBD D',
 'e(predict)